In [3]:
#%tensorflow_version 1.x

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20160704)
tf.set_random_seed(20160704)

"""**[CNN-02]** MNIST 데이터 세트를 준비한다."""

mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

type(mnist)

"""**[CNN-03]** 첫 번째 단계의 합성곱 필터와 풀링 계층을 정의한다."""

num_filters1 = 32

x = tf.placeholder(tf.float32, [None, 784], name="x")     # 28 x 28
x_image = tf.reshape(x, [-1,28,28,1])

W_conv1 = tf.Variable(tf.truncated_normal([5,5,1,num_filters1],
                                          stddev=0.1))
#h_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1,1,1,1], padding='SAME')
h_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1,1,1,1], padding='SAME', name='h_conv1')


b_conv1 = tf.Variable(tf.constant(0.1, shape=[num_filters1]))
#h_conv1_cutoff = tf.nn.relu(h_conv1 - b_conv1) 
h_conv1_cutoff = tf.nn.relu(h_conv1 - b_conv1, name='h_conv1_cutoff') 


h_pool1 = tf.nn.max_pool(h_conv1_cutoff, ksize=[1,2,2,1],
                         strides=[1,2,2,1], padding='SAME')

"""**[CNN-04]** 두 번째 단계의 합성곱 필터와 풀링 계층을 정의한다."""

num_filters2 = 64

W_conv2 = tf.Variable(
            tf.truncated_normal([5,5,num_filters1,num_filters2],
                                stddev=0.1))
#h_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1,1,1,1], padding='SAME')
h_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1,1,1,1], padding='SAME', name='h_conv2')

b_conv2 = tf.Variable(tf.constant(0.1, shape=[num_filters2]))
#h_conv2_cutoff = tf.nn.relu(h_conv2 - b_conv2)
h_conv2_cutoff = tf.nn.relu(h_conv2 - b_conv2, name='h_conv2_cutoff')


h_pool2 = tf.nn.max_pool(h_conv2_cutoff, ksize=[1,2,2,1],
                         strides=[1,2,2,1], padding='SAME')

"""**[CNN-05]** 전 결합층, 드롭아웃 계층, 소프트맥스 함수를 정의한다."""

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*num_filters2])

num_units1 = 7*7*num_filters2
num_units2 = 1024

w2 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))
b2 = tf.Variable(tf.constant(0.1, shape=[num_units2]))
hidden2 = tf.nn.relu(tf.matmul(h_pool2_flat, w2) + b2)

keep_prob = tf.placeholder(tf.float32, name="keep_prob")
hidden2_drop = tf.nn.dropout(hidden2, keep_prob)

w0 = tf.Variable(tf.zeros([num_units2, 10]))
b0 = tf.Variable(tf.zeros([10]))
p = tf.nn.softmax(tf.matmul(hidden2_drop, w0) + b0)
p = tf.identity(p, "p")

"""**[CNN-06]** 오차 함수 loss, 트레이닝 알고리즘 train_step, 정답률 accuracy을 정의한다."""

t = tf.placeholder(tf.float32, [None, 10])
#loss = -tf.reduce_sum(t * tf.log(p))
loss =  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=p, labels=tf.stop_gradient(t)))
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))





Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:
"""**[CNN-07]** 세션을 준비하고 Variable을 초기화한다."""

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

"""**[CNN-08]** 파라미터 최적화를 20000회 반복한다.

최종적으로 테스트 세트에 대해 약 99%의 정답률을 얻을 수 있다.
"""

batch_size = 100
num_epochs = 20
num_iterations = int(mnist.train.num_examples / batch_size)
print("num_iterations = %d" %(num_iterations))

epoch = 0
i = 0
for _ in range(num_epochs):
    epoch += 1
    avg_loss = 0 
    avg_accuracy = 0
    
    for iteration in range (num_iterations):
      i += 1
      batch_xs, batch_ts = mnist.train.next_batch(batch_size)
      _, loss_val, acc_val = sess.run([train_step, loss, accuracy], feed_dict={x: batch_xs, t: batch_ts, keep_prob:0.5})
      avg_loss += loss_val / num_iterations
      avg_accuracy += acc_val / num_iterations
      
      if i % 500 == 0:
          loss_val, acc_val = sess.run([loss, accuracy], 
                                        feed_dict={x:mnist.test.images, 
                                                   t:mnist.test.labels, 
                                                   keep_prob:1.0})
          print ('==> Step: %d, (Test Data) Loss: %f, Accuracy: %f' % (i, loss_val, acc_val))
          saver.save(sess, 'data/cnn_session', global_step=i)
      
    print("Epoch  :  %04d, (train Data) Loss: %0.9f,  Accuracy: %0.9f" %(epoch, avg_loss, avg_accuracy))
    
print("Learning Finished")
  
#Test model and check accuracy
print( " Accuracy for Test Data:", sess.run(accuracy, feed_dict={x:mnist.test.images, t:mnist.test.labels, keep_prob:1.0}))


num_iterations = 550
==> Step: 500, (Test Data) Loss: 1.511632, Accuracy: 0.955600
Epoch  :  0001, (train Data) Loss: 1.607454571,  Accuracy: 0.877527274


KeyboardInterrupt: 

In [ ]:
'''
ls

"""**[CNN-09]** 세션 정보를 저장한 파일이 생성되고 있음을 확인한다."""

#@title
!ls -l data/cnn_session*

cd data
|

!cd data

!ls

!cd data/

!ls

files.download("data/cnn_session-10000.data-00000-of-00001")
'''